In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install fuzzywuzzy


In [4]:
import nltk
import random
import csv
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from fuzzywuzzy import fuzz

# Download necessary NLTK resources
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Define intents and responses
intents = {
    'greeting': {
        'patterns': ['hi','hello','hey','greetings','what\'s up','howdy','good morning','good evening','how are you?'],
        'responses': [
            'Hello! How can I help you today?'
            'Hi there! What would you like to know?',
            'Greetings! I\'m here to assist you with any questions.',
            'Good day! How can I assist you?',
            'Hey! What’s on your mind?',
            'I\'m doing well, thank you! How about you?'
        ]
    },
    'job_openings': {
        'patterns': [
            'job openings', 'available positions', 'job listings',
            'career opportunities', 'current vacancies', 'open positions',
            'job search', 'job opportunities', 'job application',
            'hiring now', 'job vacancies','where can i find jobs',
        ],
        'responses': [
            'To find job openings, I recommend checking popular job boards like LinkedIn, Indeed, or Glassdoor.',
            'You can also visit company websites for their career pages to find available positions.',
            'Don\'t forget to network! Sometimes the best opportunities come from connections.',
            'You can set up job alerts on job boards to receive notifications for new openings that match your criteria.',
            'Consider using recruitment agencies that specialize in your field for more tailored job opportunities.',
            'Networking events and job fairs can also be great places to discover job openings.'

        ]
    },
    'skill_requirements': {
        'patterns': [
            'skill requirements', 'required skills', 'what skills do I need',
            'skill set', 'skills needed', 'skills for jobs',
            'skills needed for [specific job]', 'what skills are in demand',
            'essential skills for [specific field]'

        ],
        'responses': [
            'Skill requirements vary depending on the job. Common skills include communication, problem-solving, and teamwork.',
            'For specific roles, you might need technical skills like programming languages or data analysis.',
            'Consider researching job descriptions in your field to identify key skills.',
            'In addition to technical skills, soft skills like adaptability and emotional intelligence are becoming increasingly important.',
            'Look for online resources or industry reports that highlight trending skills in your field.',
            'Certifications can also enhance your qualifications for specific roles.'
        ]
    },
    'resume_building': {
        'patterns': [
            'resume building', 'how to write a resume', 'resume tips',
            'CV advice', 'creating a resume', 'resume format',
            'resume examples', 'how to update my resume',
            'resume mistakes to avoid'
        ],
        'responses': [
            'When building a resume, focus on your relevant skills and experiences.',
            'Use action verbs, quantify your achievements, and tailor your resume to the job description.',
            'Keep it concise and well-formatted. Consider using templates available online.',
            'Make sure to customize your resume for each job application to highlight the most relevant experience.',
            'Avoid common mistakes like typos or using an unprofessional email address.'
        ]
    },
    'skill_improvement': {
        'patterns': [
            'improve skills', 'skill development', 'learn new skills',
            'upgrade skillset', 'how to learn skills', 'best ways to learn skills',
            'resources for skill development', 'how to practice skills',
            'mentorship opportunities'
        ],
        'responses': [
            'To improve your skills, consider taking online courses, attending workshops, or working on personal projects.',
            'Platforms like Coursera, edX, and Udemy offer a wide range of courses to enhance your skillset.',
            'Joining community groups or forums can also provide valuable resources and support.',
            'Consider finding a mentor in your field who can provide guidance and resources for skill development.',
            'Participating in hackathons or competitions can also be a fun way to improve your skills while networking.'
        ]
    },
    'interview_tips': {
        'patterns': [
            'interview tips', 'how to prepare for an interview',
            'interview advice', 'succeed in interviews',
            'common interview questions', 'how to answer behavioral questions',
            'what to wear to an interview', 'follow-up after interview'
        ],
        'responses': [
            'Research the company and the role you\'re applying for .',
            'Practice common interview questions and prepare your own questions to ask the interviewer.',
            'dress appropriately and arrive on time. Confidence and good communication are key!',
            'Prepare examples of your past experiences that demonstrate your skills and achievements.',
            'Sending a thank-you email after the interview can leave a positive impression on the interviewer.',
            ' wear a business professional attire'
        ]
    },
    'salary_negotiation': {
        'patterns': [
            'salary negotiation', 'how to negotiate salary',
            'salary tips', 'negotiating a raise',
            'salary expectations', 'what to say during salary negotiation'
        ],
        'responses': [
            'Research the market rate for your role to determine a fair salary range.',
            'Be confident and assertive during the negotiation, but also be open to compromise.',
            'Highlight your achievements and the value you bring to the company.',
            'Consider factors beyond salary, such as benefits, vacation time, or professional development opportunities.',
            'Be prepared to explain why you\'re worth the salary you\'re asking for.'
        ]
    },
    'course_recommendation': {
        'patterns': ['recommend a course', 'suggest a course', 'what course should I take', 'course recommendation'],
        'responses': ['Based on your interests, I\'d recommend the following course: {course_name}. You can find it here: {course_url}']
    },
    'skill_based_course': {
        'patterns': ['course for {skill}', 'learn {skill}', 'improve my {skill}'],
        'responses': ['For {skill}, I\'d recommend the course: {course_name}. You can find it here: {course_url}']
    }
}


# Function to tokenize and lemmatize input text
def tokenize_and_lemmatize(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

# Function to match user input with intents
def match_intent(user_input):
    tokens = tokenize_and_lemmatize(user_input)
    for intent, data in intents.items():
        for pattern in data['patterns']:
            pattern_tokens = tokenize_and_lemmatize(pattern)
            if all(token in tokens for token in pattern_tokens):
                return intent
    return None

# Function to find a course based on skill or any other input
def find_course(input_text):
    matching_courses = []
    input_text = input_text.lower()

    # Check each course for skill matches
    for course_id, course in courses.items():
        # Convert skills to lowercase for case-insensitive matching
        skills = [skill.lower().strip() for skill in course['job_skills']]

        # Calculate skill match similarity
        skill_match = False
        for skill in skills:
            # Check if input text contains any of the skills
            if (input_text in skill) or (skill in input_text) or (fuzz.ratio(input_text, skill) > 70):
                skill_match = True
                break

        if skill_match:
            similarity = fuzz.ratio(input_text, course['course_name'].lower()) * 0.3 + \
                        max([fuzz.ratio(input_text, skill.lower()) for skill in course['job_skills']]) * 0.7

            matching_courses.append({
                'course_name': course['course_name'],
                'course_url': course['course_url'],
                'relevance': similarity/100,
                'skills': course['job_skills']
            })

    # Sort matches by relevance score
    matching_courses.sort(key=lambda x: x['relevance'], reverse=True)

    # Return top 3 most relevant matches
    return matching_courses[:3]

def respond(user_input):
    intent = match_intent(user_input)

    if intent in ['course_recommendation', 'skill_based_course'] or 'course' in user_input.lower():
        matching_courses = find_course(user_input)
        if matching_courses:
            response = "Here are some relevant course recommendations based on your skills interest:\n\n"
            for i, course in enumerate(matching_courses, 1):
                response += f"{i}. {course['course_name']}\n"
                response += f"   Relevance Score: {course['relevance']:.2f}\n"
                response += f"   Course Link: {course['course_url']}\n"
                response += f"   Related Skills: {', '.join(course['skills'])}\n\n"
            return response
        else:
            return "I couldn't find any courses matching your criteria. Try searching for a different skill or topic."
    elif intent:
        return random.choice(intents[intent]['responses'])
    else:
        return "I'm not sure what you're looking for. Try asking about specific skills or courses you're interested in."
# Load courses data
courses = {}
with open('/content/drive/MyDrive/Colab Notebooks/job_dataset', 'r') as file:
    reader = csv.reader(file)
    headers = next(reader)  # Read the header row
    for row in reader:
        course_id = row[0]
        courses[course_id] = {
            'job_skills': row[7].split(','),
            'job_type': row[5],
            'job_category': row[6],
            'job_roles': row[8].split(','),
            'course_name': row[9],
            'course_url': row[10]
        }


# Run the chatbot
if __name__ == "__main__":
    print("Chatbot: Hello! I'm here to help you with job-related queries and course recommendations. How can I assist you today?")

    while True:
        user_input = input("You: ")

        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("Chatbot: Goodbye! Have a great day!")
            break

        response = respond(user_input)
        print("Chatbot:", response)

Chatbot: Hello! I'm here to help you with job-related queries and course recommendations. How can I assist you today?
You: hi
Chatbot: Hello! How can I help you today?Hi there! What would you like to know?
You: can you suggest a course for excel
Chatbot: Here are some relevant course recommendations based on your skills interest:

1. Nutrición y obesidad: control de sobrepeso
   Relevance Score: 0.32
   Course Link: https://www.coursera.org/learn/nutricion-obesidad-sobrepeso
   Related Skills: Data Analysis, Excel, statistical modelling

2. Connect an App to a Cloud SQL for PostgreSQL Instance
   Relevance Score: 0.32
   Course Link: https://www.coursera.org/learn/googlecloud-connect-an-app-to-a-cloud-sql-for-postgresql-instance-mhiqu?specialization=google-cloud-database-engineer
   Related Skills: Financial analysis, Excel, accounting

3. Securing a Cloud SQL for PostgreSQL Instance
   Relevance Score: 0.32
   Course Link: https://www.coursera.org/learn/googlecloud-securing-a-cloud-sq

In [ ]:
import pandas as pd
import random
from fuzzywuzzy import fuzz

class JobCourseBot:
    def __init__(self, data_file):
        self.df = pd.read_csv(data_file)
        self.job_categories = self.df['Job_Category'].unique().tolist()
        self.skills_map = {job: self.df[self.df['Job_Category'] == job]['Required_Skills'].iloc[0].split(',') for job in self.job_categories}
        self.employment_types = self.df['Job_Type'].unique().tolist()

    def greet(self):
        return """Hello! I'm your job and course recommendation assistant. I can help you with:
1. Job recommendations based on skills
2. Course recommendations for specific careers or skills
3. Skill requirements for different roles
4. Employment type information
5. Interview tips
6. Resume building advice

What would you like to know about?"""

    def get_job_recommendations(self, skills):
        matching_jobs = []
        skills_lower = [skill.lower() for skill in skills]

        for job, required_skills in self.skills_map.items():
            required_lower = [skill.lower() for skill in required_skills]
            if any(skill in required_lower for skill in skills_lower):
                matching_jobs.append(job)

        if matching_jobs:
            return f"Based on your skills, you might be interested in these roles: {', '.join(matching_jobs)}"
        return "I couldn't find specific job matches. Please try with different skills."

    def get_required_skills(self, job_category):
        job_category = job_category.title()
        if job_category in self.skills_map:
            skills = self.skills_map[job_category]
            return f"For {job_category}, the key required skills are: {', '.join(skills)}"
        return "I couldn't find that job category. Please try with one of: " + ", ".join(self.job_categories)

    def get_course_recommendations(self, keywords):
        keywords = keywords.lower().split()
        relevant_courses = []

        for _, row in self.df.iterrows():
            course_name = row['Title'].lower()
            course_url = row['URL'].lower()

            # Calculate the score based on the keywords in Title and URL
            title_score = max(fuzz.partial_ratio(keyword, course_name) for keyword in keywords)
            url_score = max(fuzz.partial_ratio(keyword, course_url) for keyword in keywords)

            # Combine scores to get a final score
            total_score = title_score + url_score

            # Check if the total score exceeds the threshold
            if total_score > 120:  # Adjust this threshold as needed
                relevant_courses.append((row['Title'], row['URL'], total_score))

        # Sort the relevant courses by score in descending order
        relevant_courses.sort(key=lambda x: x[2], reverse=True)
        top_courses = relevant_courses[:5]  # Get the top 5 courses

        # Prepare the result for output
        if top_courses:
            result = "Recommended courses based on your query:\n"
            for course, url, score in top_courses:
                result += f"- {course}\n  URL: {url}\n  Relevance Score: {score}\n"
            return result
        return "I couldn't find any relevant courses. Please try with different keywords."
    def get_interview_tips(self):
        tips = [
            "Research the company thoroughly before the interview.",
            "Prepare specific examples to demonstrate your skills and experience.",
            "Practice common interview questions with a friend or family member.",
            "Dress professionally and arrive early to the interview.",
            "Bring extra copies of your resume and a notepad.",
            "Ask thoughtful questions about the company and role.",
            "Follow up with a thank-you email after the interview.",
            "Be prepared to discuss your strengths and weaknesses.",
            "Use the STAR method (Situation, Task, Action, Result) to answer behavioral questions.",
            "Maintain good eye contact and positive body language throughout the interview."
        ]
        return "Here are some interview tips:\n" + "\n".join(f"- {tip}" for tip in random.sample(tips, 5))

    def get_resume_tips(self):
        tips = [
            "Tailor your resume to the specific job you're applying for.",
            "Use action verbs to describe your accomplishments.",
            "Quantify your achievements with specific numbers and metrics.",
            "Keep your resume concise and limit it to 1-2 pages.",
            "Use a clean, professional format with consistent fonts and spacing.",
            "Include a strong summary or objective statement at the top.",
            "List your work experience in reverse chronological order.",
            "Proofread carefully for any spelling or grammatical errors.",
            "Include relevant skills and certifications.",
            "Use keywords from the job description in your resume."
        ]
        return "Here are some resume building tips:\n" + "\n".join(f"- {tip}" for tip in random.sample(tips, 5))

    def get_job_openings(self, job_category):
        job_category = job_category.title()
        openings = self.df[(self.df['Job_Category'] == job_category) & (self.df['Candidate_Work_Experience'] > 0)]

        if not openings.empty:
            result = f"Current job openings for {job_category}:\n"
            for _, row in openings.iterrows():
                result += f"- {row['Job_Category']} ({row['Job_Type']}): {row['Candidate_Work_Experience']} years of experience required\n"
            return result
        return f"I couldn't find any current openings for {job_category}. Please check back later or try a different category."

    def process_query(self, query):
        query = query.lower()

        if "skill" in query or "skills" in query:
            for job in self.job_categories:
                if job.lower() in query:
                    return self.get_required_skills(job)

        elif "course" in query or "learn" in query or "study" in query:
            return self.get_course_recommendations(query)

        elif "job" in query or "career" in query or "work" in query:
            if "opening" in query or "available" in query:
                for job in self.job_categories:
                    if job.lower() in query:
                        return self.get_job_openings(job)
            else:
                mentioned_skills = []
                for job, skills in self.skills_map.items():
                    for skill in skills:
                        if skill.lower() in query:
                            mentioned_skills.append(skill)
                if mentioned_skills:
                    return self.get_job_recommendations(mentioned_skills)

        elif "interview" in query and "tip" in query:
            return self.get_interview_tips()

        elif "resume" in query or "cv" in query:
            return self.get_resume_tips()

        return """I'm not sure what you're asking. You can ask me about:
- Required skills for a specific job
- Course recommendations for a career or skill
- Job recommendations based on skills
- Job openings for a specific category
- Interview tips
- Resume building advice
Please try rephrasing your question."""

# Example usage:
bot = JobCourseBot('/content/drive/MyDrive/Colab Notebooks/job_dataset')  # Replace with your actual CSV file path
print(bot.greet())

# Interactive loop
def chat():
    bot = JobCourseBot('/content/drive/MyDrive/Colab Notebooks/job_dataset')  # Replace with your actual CSV file path
    print(bot.greet())

    while True:
        user_input = input("\nYour question (or 'quit' to exit): ")
        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("Goodbye! Have a great day!")
            break

        response = bot.process_query(user_input)
        print("\nBot:", response)

# Run the chat
if __name__ == "__main__":
    chat()

Hello! I'm your job and course recommendation assistant. I can help you with:
1. Job recommendations based on skills
2. Course recommendations for specific careers or skills
3. Skill requirements for different roles
4. Employment type information
5. Interview tips
6. Resume building advice

What would you like to know about?
Hello! I'm your job and course recommendation assistant. I can help you with:
1. Job recommendations based on skills
2. Course recommendations for specific careers or skills
3. Skill requirements for different roles
4. Employment type information
5. Interview tips
6. Resume building advice

What would you like to know about?

Your question (or 'quit' to exit): gine interview tips

Bot: Here are some interview tips:
- Prepare specific examples to demonstrate your skills and experience.
- Be prepared to discuss your strengths and weaknesses.
- Research the company thoroughly before the interview.
- Dress professionally and arrive early to the interview.
- Use the STA